# 作業

實作各種不一樣的方式來調整飽和 / 對比 / 明亮

1. 改變 HSL color space 來調整飽和度
2. 實作直方圖均衡
3. alpha/ beta 調整對比 / 明亮

In [4]:
import cv2
import numpy as np

img_path = 'D:/1st-DL-CVMarathon/homework/data/HW001/lena.png'

# 以彩色圖片的方式載入
img = cv2.imread(img_path, cv2.IMREAD_COLOR)
print(img)

[[[125 137 226]
  [125 137 226]
  [133 137 223]
  ...
  [122 148 230]
  [110 130 221]
  [ 90  99 200]]

 [[125 137 226]
  [125 137 226]
  [133 137 223]
  ...
  [122 148 230]
  [110 130 221]
  [ 90  99 200]]

 [[125 137 226]
  [125 137 226]
  [133 137 223]
  ...
  [122 148 230]
  [110 130 221]
  [ 90  99 200]]

 ...

 [[ 60  18  84]
  [ 60  18  84]
  [ 58  27  92]
  ...
  [ 84  73 173]
  [ 76  68 172]
  [ 79  62 177]]

 [[ 57  22  82]
  [ 57  22  82]
  [ 62  32  96]
  ...
  [ 79  70 179]
  [ 81  71 181]
  [ 81  74 185]]

 [[ 57  22  82]
  [ 57  22  82]
  [ 62  32  96]
  ...
  [ 79  70 179]
  [ 81  71 181]
  [ 81  74 185]]]


## 改變飽和度

- 轉換成 HSV color space, 改變 s channel 的值

In [11]:
# 為了要改變飽和度，我們先把 color space 轉成 HSV 格式
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#print(img_hsv)
# 操做飽和度時候需要轉成百分比(小數點)
change_percentage = 0.2
img_hsv_down=img_hls.astype('float32') #float32:從整數變成有小數點的格式
#print(img_hls_down)
# Nelson: 轉成小數點並調降成兩成飽和度
img_hsv_down[..., 1] = img_hsv_down[..., 1] / 255 - change_percentage

# 在 HSV color space 增加飽和度
# Nelson: 操作飽和度時候需要轉成百分比(小數點)
img_hsv_up = img_hsv.astype("float32")
# Nelson: 轉成小數點並增加成兩成飽和度
img_hsv_up[..., 1] = img_hsv_up[..., 1] / 255 + change_percentage

# 轉換 color space 回 BGR
# Nelson: 後面再做一些邊緣條件判斷, 確保所有值都在合理範圍(0~1)
img_hsv_down[img_hsv_down[..., 1] < 0] = 0
img_hsv_up[img_hsv_up[..., 1] > 1] = 1
# Nelson: 再把圖片依序轉回0~255區間 > uint8格式 > BGR格式 就可以顯示圖片了
img_hsv_down[..., 1] = img_hsv_down[..., 1] * 255
img_hsv_down = img_hsv_down.astype("uint8")
img_hsv_down = cv2.cvtColor(img_hsv_down, cv2.COLOR_HSV2BGR)
img_hsv_up[..., 1] = img_hsv_up[..., 1] * 255
img_hsv_up = img_hsv_up.astype("uint8")
img_hsv_up = cv2.cvtColor(img_hsv_up, cv2.COLOR_HSV2BGR)

while True:
    cv2.imshow('img', img)    
    cv2.imshow('img_hsv', img_hsv)
    cv2.imshow('img_hsv_down', img_hsv_down)
    cv2.imshow('img_hsv_up', img_hsv_up)   
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

## 直方圖均衡

1. case 1: 把彩圖拆開對每個 channel 個別做直方圖均衡再組合起來
2. case 2: 轉換 color space 到 HSV 之後對其中一個 channel 做直方圖均衡

In [13]:
# HW...
# case 1
# 每個 channel 個別做直方圖均衡
# Nelson: https://stackoverflow.com/questions/31998428/opencv-python-equalizehist-colored-image/43063287#43063287
equalHist_by_channel = (cv2.equalizeHist(img[:, :, 0]), cv2.equalizeHist(img[:, :, 1]), cv2.equalizeHist(img[:, :, 2]))

# HW...
# 組合經過直方圖均衡的每個 channel
img_bgr_equal = cv2.merge(equalHist_by_channel)

# HW...
# case 2 - 轉換 color space 後只對其中一個 channel 做直方圖均衡
img_hsv_equal = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
H, S, V = cv2.split(img_hsv_equal)
img_hsv_equal[:, :, 2] = cv2.equalizeHist(V)
img_hsv_equal = cv2.cvtColor(img_hsv_equal, cv2.COLOR_HSV2BGR)

# img_hsv_equal = img_hsv.copy()
# img_hsv_equal[..., 2] = cv2.equalizeHist(img_hsv[..., 2])
# img_hsv_equal = cv2.cvtColor(img_hsv_equal, cv2.COLOR_HSV2BGR)

# 組合圖片 + 顯示圖片
img_bgr_equalHist = np.hstack((img, img_bgr_equal, img_hsv_equal))
while True:
    # 比較 (原圖, BGR color space 對每個 channel 做直方圖均衡, HSV color space 對明度做直方圖均衡)
    cv2.imshow('bgr equal histogram', img_bgr_equalHist)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

## 調整對比 / 明亮

In [12]:
# HW...
# alpha: 控制對比度 (1.0~3.0)
# beta: 控制明亮度 (0~255)
add_contrast = cv2.convertScaleAbs(img, alpha=2.0, beta=0)
add_lighness = cv2.convertScaleAbs(img, alpha=1.0, beta=100)

# 組合圖片 + 顯示圖片
img_contrast_light = np.hstack((img, add_contrast, add_lighness))
while True:
    # 比較不同程度的對比 / 明亮
    cv2.imshow('adjust contrast and brighness', img_contrast_light)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break